# Transformations and Actions

Es gibt nur diese zwei verschiedene Operationen auf RDDs. Es ist wichtig, dass wir diese verstehen und auseinanderhalten können.

## Transformationen

* Eine Operation welche aus einem RDD einen neuen RDD erzeugt.
* Sie bauen den Lineage Graph auf

Bespiele sind:

* Lesen einer Datei
* Mappen einer Spalte in einen anderen Wert
* Filter eines RDDs (nur Personen aus Hamburg)

Sie werden **lazy** ausgeführt: erst wenn eine Action-Operation ausgeführt wird

![](RDD-lineage-graph.dio.svg)

## Actions

* Gibt das Ergebniss von RDD-Anwendungen aus
* Stößt die Anwendung von Transformationen über den Lineage-Graph an

Bespiele sind:

* Schreibe eine Ausgabe auf die Festplatte
* Gebe die Ausgabe auf dem Bildschirm aus
* Gib die Anzahl aus


In [3]:
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = (
    SparkSession
        .builder
        .appName("RDD-Basic-Operations")
        .master("local[4]")
        .getOrCreate()
)

sc = spark.sparkContext

spark

23/09/09 00:58:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
# erzeuge RDD mit Hilfe des Spark Kontext

numbers_rdd = sc.parallelize([1, 2, 3, 4, 5])

Wurde die Operation wirklich ausgeführt?

Nein: sie wurde nur dem Lineage Graph hinzugefügt.

Wir können das in der Spark-UI überprüfen.

In [7]:
# Nun geben wir den RDD aus

numbers_rdd.collect()

[1, 2, 3, 4, 5]

In der Spark-UI sehen wir nun **1** Job und **4** Tasks.

In [8]:
# Nun das erste Element ausgeben

numbers_rdd.first()

1

Wir sehen in der Spark UI einen neuen Job mit genau einem Task.

In [12]:
# Nun führen wir mehere Operationen hintereinander aus

taxi_zones_rdd = sc.textFile("TaxiZones.csv")

# aufsplitten
taxi_zones_with_cols_rdd = taxi_zones_rdd.map(lambda zone: zone.split(","))

# Pair RDD über die Bezirke

taxi_zones_by_bezirk = taxi_zones_with_cols_rdd.map(lambda zone: (zone[1],1))

# aufsummieren
taxi_zones_count_bezirke = taxi_zones_by_bezirk.reduceByKey( lambda a, b: a + b)

# filtern

only_big_bezirke = taxi_zones_count_bezirke.filter( lambda row: row[1] > 10 )


In [13]:
only_big_bezirke.collect()

[('Manhattan', 69),
 ('Brooklyn', 61),
 ('Queens', 69),
 ('Bronx', 43),
 ('Staten Island', 20)]

Nun in der Spark-UI prüfen was passiert ist...

## Narrow Transformations

Wir erinnern uns: es gibt zwei Arten von Transformationen. Narrow und Wide. Wir schauen und nun Narrow Transformations an.

**Definition**

Eine Narrow Transformation ist eine Transformation mit der für eine Output-Partition nur genau eine Eingabe Partition genutzt wird.

![](RDD-narrow.dio.svg)

Beispiele:
* Map
* MapPartition
* Filter
* Sample
* Union
* ...

**Eigenschaften**

* Die Zahl der Eingabe Partitionen ist in der Regel gleich der der Ausgabe-Partitionen.
* Sehr schnell. Zwischen den Partitionen werden keine 

## Wide Transformation


**Definition**

Eine Wide-Transformationen besteht aus zwei Schritten und führt eine sogenannte Shuffle-Partition ein.

![](RDD-wide.dio.svg)



# Jobs, Stages and Tasks

## Setting up Spark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("jobs-stages-tasks").master("local[4]").getOrCreate()
sc = spark.sparkContext
spark

23/09/08 18:00:52 WARN Utils: Your hostname, pupil-a resolves to a loopback address: 127.0.1.1; using 167.235.141.210 instead (on interface eth0)
23/09/08 18:00:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/08 18:00:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/08 18:00:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/08 18:00:54 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Reading the file

In [26]:
zones_with_cols_rdd = sc.textFile("TaxiZones.csv", 4)
zones_with_cols_rdd.take(10)

['1,EWR,Newark Airport,EWR',
 '2,Queens,Jamaica Bay,Boro Zone',
 '3,Bronx,Allerton/Pelham Gardens,Boro Zone',
 '4,Manhattan,Alphabet City,Yellow Zone',
 '5,Staten Island,Arden Heights,Boro Zone',
 '6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone',
 '7,Queens,Astoria,Boro Zone',
 '8,Queens,Astoria Park,Boro Zone',
 '9,Queens,Auburndale,Boro Zone',
 '10,Queens,Baisley Park,Boro Zone']

Überlege wieviele
 * Jobs
 * Transformationen
 * Actions
 * Stages
erzeugt wurden?

## Reading and splitting

In [11]:
# Reading into 4 partitions 
zones_rdd = sc.textFile("TaxiZones.csv", 4)
# Splitting each row by ,
zones_with_cols_rdd = zones_rdd.map(lambda x: x.split(","))

zones_with_cols_rdd.take(10)

[['1', 'EWR', 'Newark Airport', 'EWR'],
 ['2', 'Queens', 'Jamaica Bay', 'Boro Zone'],
 ['3', 'Bronx', 'Allerton/Pelham Gardens', 'Boro Zone'],
 ['4', 'Manhattan', 'Alphabet City', 'Yellow Zone'],
 ['5', 'Staten Island', 'Arden Heights', 'Boro Zone'],
 ['6', 'Staten Island', 'Arrochar/Fort Wadsworth', 'Boro Zone'],
 ['7', 'Queens', 'Astoria', 'Boro Zone'],
 ['8', 'Queens', 'Astoria Park', 'Boro Zone'],
 ['9', 'Queens', 'Auburndale', 'Boro Zone'],
 ['10', 'Queens', 'Baisley Park', 'Boro Zone']]

In [12]:
# Hat sich die Anzahl der Partitionen im Laufe der Transformationen verändert?

print("After reading the file: " + str(zones_rdd.getNumPartitions()))
print("After applying map: " + str(zones_with_cols_rdd.getNumPartitions()))

After reading the file: 4
After applying map: 4


## Reading and creating a pair RDD

In [28]:
# Reading into 4 partitions 
zones_rdd = sc.textFile("TaxiZones.csv", 4)
# Splitting each row by ,
zones_with_cols_rdd = zones_rdd.map(lambda x: x.split(","))

# Now creating a pair RDD for counting the amount of boroughs
zones_pair_rdd = zones_with_cols_rdd.map(lambda x: (x[0],1))

print(zones_rdd.count())
print(zones_with_cols_rdd.count())
print(zones_pair_rdd.count())

zones_with_cols_rdd.take(10)

zones_pair_rdd.take(2)

265


265


265


[('1', 1), ('2', 1)]

### Now also finding distinct records

In [14]:
# Reading into 4 partitions 
zones_rdd = sc.textFile("TaxiZones.csv", 4)
# Splitting each row by ,
zones_with_cols_rdd = zones_rdd.map(lambda x: x.split(","))

# Now creating a pair RDD for counting the amount of boroughs
zones_pair_rdd = zones_with_cols_rdd.map(lambda x: (x[1],1))

distinct_zones_pair_rdd = zones_pair_rdd.distinct()

print(zones_rdd.count())
print(zones_with_cols_rdd.count())
print(zones_pair_rdd.count())

distinct_zones_pair_rdd.collect()

265


265


265


[('Bronx', 1),
 ('Staten Island', 1),
 ('Queens', 1),
 ('EWR', 1),
 ('Manhattan', 1),
 ('Brooklyn', 1),
 ('Unknown', 1)]

In [29]:
# Reading into 4 partitions 
zones_rdd = sc.textFile("TaxiZones.csv", 4)
# Splitting each row by ,
zones_with_cols_rdd = zones_rdd.map(lambda x: x.split(","))

# Now creating a pair RDD for counting the amount of boroughs
zones_pair_rdd = zones_with_cols_rdd.map(lambda x: (x[1],1))

counted = zones_pair_rdd.reduceByKey(lambda x, y: x + y)

counted.take(10)

[('Bronx', 43),
 ('Staten Island', 20),
 ('EWR', 1),
 ('Manhattan', 69),
 ('Brooklyn', 61),
 ('Unknown', 2),
 ('Queens', 69)]

In [19]:
zones_pair_rdd.getNumPartitions()

4